In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
from collections import OrderedDict

In [2]:
url = 'https://www.americanas.com.br/busca/geladeira'

In [3]:
headers = {'user-agent': 'Chrome/88.0.4324.150'}
raw = requests.get(url, headers=headers).content
soup = BeautifulSoup(raw)

In [4]:
produtos = soup.find_all('div', {'class':'src__Wrapper-sc-1k0ejj6-2 dGIFSc'})

In [5]:
url_produtos = []
for produto in produtos:
    url_produtos.append('https://www.submarino.com.br' + produto.a['href'])

In [6]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time

In [7]:
headers_list = [
    # Firefox 77 Mac
     {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Firefox 77 Windows
    {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "Referer": "https://www.google.com/",
        "DNT": "1",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 83 Mac
    {
        "Connection": "keep-alive",
        "DNT": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Chrome 83 Windows 
    {
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-User": "?1",
        "Sec-Fetch-Dest": "document",
        "Referer": "https://www.google.com/",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9"
    }
]

ordered_headers_list = []
for headers in headers_list:
    h = OrderedDict()
    for header,value in headers.items():
        h[header]=value
    ordered_headers_list.append(h)

In [8]:
imagens = []
names = []
seller_description = []
seller_url = []
price = []
reviews_text = []
i = 0
for url in url_produtos:
    while True:
        headers = random.choice(headers_list)
        try:
            raw_product = requests.get(url, headers=headers).content
            time.sleep(2)
            soup_product = BeautifulSoup(raw_product)
            names.append(soup_product.find_all('h1',{'id':'product-name-default'})[0].text)
            imagens.append(soup_product.find_all('picture',{'class':'Picture-sc-1m5klhe-2 hXieYa'})[0].img['src'])
            price.append(soup_product.find_all('span',{'class':'price__SalesPrice-sc-1i11rkh-2 jjADsQ TextUI-sc-12tokcy-0 CIZtP'})[0].text)
            chromeOptions = Options()
            chromeOptions.add_argument("--start-maximized")
            driver = webdriver.Chrome(options=chromeOptions)
            driver.implicitly_wait(5)
            driver.get(url)
            while True:
                try:
                    x_path_button = '/html/body/div[4]/div/div/div[8]/div/section/div/section/div/button/div/span'
                    time.sleep(3)
                    botao = driver.find_element_by_xpath(x_path_button)
                    driver.execute_script("window.scrollTo(0, {});".format(botao.location['y']-400))
                    time.sleep(3)
                    action=ActionChains(driver)
                    action.move_to_element(botao).perform()
                    time.sleep(3)
                    botao.click()
                except Exception as e:
                    print(e)
                    print('End of comments')
                    break
            raw = driver.page_source
            soup = BeautifulSoup(raw)
            reviews = soup.find_all('ul',{'class':'ListUlUI-sc-13efbnv-0 DbZws'})[0].find_all('div',{'class':'review-desc ViewUI-sc-1ijittn-6 iXIDWU'})
            reviews_text.append([review.text for review in reviews])
            seller_soup = soup.find_all('div',{'class':'SpacingUI-pvph4q-0 zvGsc ViewUI-sc-1ijittn-6 iXIDWU'})[0]
            if seller_soup.a == None:
                seller_description.append('Submarino')
                seller_url.append('')
            else:
                seller_description.append(seller_soup.a.text)
                seller_url.append(seller_soup.a['href'])
            driver.quit()
            print('Product {} processed'.format(names[-1]))
            time.sleep(2)
        except:
            print('Scrapping error. Trying again...')
            driver.quit()
            time.sleep(random.randint(120,240))
            continue
        break

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div/div/div[8]/div/section/div/section/div/button/div/span"}
  (Session info: chrome=88.0.4324.182)

End of comments
Product Geladeira/Refrigerador Electrolux Duplex TW42S Top Freezer com Dispenser De Água 382L Inox - 220V processed
Message: move target out of bounds
  (Session info: chrome=88.0.4324.182)

End of comments
Product Geladeira/Refrigerador Brastemp Frost Free BRM44 375 Litros - Evox processed
Scrapping error. Trying again...


KeyboardInterrupt: 

In [11]:
seller_url

['', '']

In [9]:
reviews_text

[['Recebi a geladeira ontem , deixei ligada a noote toda, não resfriou, nem congelador gelou .\nImpossivel falar , contactar submarino!!!\nLiga liga, e não atendem !!!\nAgora liga pra comprar atendem \nFavor entrar em contato urgente!!\nPois vai  passar a data para troca !!!',
  'Entrega mais rápida que já vi na vida ,comprei na quinta-feira entregaram no domingo sem falar que a geladeira é a melhor que já tive , muito silenciosa, espaçosa, gela muito rápido, eu super recomendo,  e a equipe submarino só tenho a dizer que estão de parabéns.',
  'Não era a que eu esperava não pensei que ela era melhor muito pequena.',
  'Gente recomendo minha primeira compra no site e eu amei chegou muito rápido',
  'A geladeira chegou no prazo. Não tem feito nenhum barulho estranho como nos outros relatos. Contudo, não tem luz no freezer. O botão para a saída de água na porta não é automatico. Considero pequena pra quem tem criança ou família grande. Congela super bem...contudo sem espaço para latinhas.

In [63]:
import pandas as pd
def converter_df(names, imagens, seller_description, seller_cnpj, seller_razao_social, original_price, promotional_price):
    data = {'name':names,
            'image':imagens,
            'seller_description':seller_description,
            'seller_cnpj':seller_cnpj,
            'seller_razao_social':seller_razao_social,
            'original_price':original_price,
            'promotional_price':promotional_price}
    df = pd.DataFrame(data)
    return df

In [73]:
df = converter_df(names, imagens, seller_description, seller_cnpj, seller_razao_social, original_price, promotional_price)

In [56]:
raw_product = requests.get('https://www.submarino.com.br/produto/130027877?pfm_carac=geladeira&pfm_index=6&pfm_page=search&pfm_pos=grid&pfm_type=search_page', headers=headers).content
# raw_product = requests.get('https://www.submarino.com.br/produto/111957489/refrigerador-1-porta-electrolux-re31-214-litros-branco?pfm_carac=geladeira&pfm_index=4&pfm_page=search&pfm_pos=grid&pfm_type=search_page&voltagem=110%20volts',headers=headers).content
soup_product = BeautifulSoup(raw_product)
image = soup_product.find_all('picture',{'class':'Picture-sc-1m5klhe-2 hXieYa'})[0].img['src']
name = soup_product.find_all('h1',{'id':'product-name-default'})[0].text
# unavailable = soup_product.find_all('div', {'class':'unavailable__product'})
seller = soup_product.find_all('div',{'class':'SpacingUI-pvph4q-0 zvGsc ViewUI-sc-1ijittn-6 iXIDWU'})
price = soup_product.find_all('span',{'class':'price__SalesPrice-sc-1i11rkh-2 jjADsQ TextUI-sc-12tokcy-0 CIZtP'})[0].text

In [40]:
price

'R$ 1.499,00'

In [57]:
seller

[<div class="SpacingUI-pvph4q-0 zvGsc ViewUI-sc-1ijittn-6 iXIDWU"></div>]

In [42]:
seller

[<div class="SpacingUI-pvph4q-0 zvGsc ViewUI-sc-1ijittn-6 iXIDWU"></div>]

In [49]:
image[0]

<picture class="Picture-sc-1m5klhe-2 hXieYa"><source media="(min-width: 1280px)" srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204SZ.jpg"/><source media="(min-width: 1025px)" srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204_1GG.jpg"/><source media="(min-width: 768px)" srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204P1.jpg"/><img alt="Foto 1 - Geladeira / Refrigerador Consul 1 Porta Facilite CRB36 com Frost Free 300L - Branco" class="Image-sc-1m5klhe-0 hrWOqf" src="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204P1.jpg"/></picture>

In [38]:
seller[0].a['href']

'/lojista/45543915000181'

In [ ]:
<div class="Wrapper-sc-1m5klhe-1 hnnyqp"><picture class="Picture-sc-1m5klhe-2 hXieYa"><source srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204SZ.jpg" media="(min-width: 1280px)"><source srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204_1GG.jpg" media="(min-width: 1025px)"><source srcset="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204P1.jpg" media="(min-width: 768px)"><img src="https://images-submarino.b2w.io/produtos/01/00/item/5091/2/5091204P1.jpg" alt="Foto 1 - Geladeira / Refrigerador Consul 1 Porta Facilite CRB36 com Frost Free 300L - Branco" class="Image-sc-1m5klhe-0 hrWOqf"></picture></div>